In [6]:
using System;
using System.Collections.Concurrent;
using System.Threading;

public interface ICommand
{
    bool Execute();
}

public interface IScheduler
{
    bool HasCommand();
    ICommand Select();
    void Add(ICommand cmd);
}

public class RoundRobinScheduler : IScheduler
{
    private ConcurrentQueue<ICommand> commandQueue = new ConcurrentQueue<ICommand>();
    private int currentIndex = 0;

    public bool HasCommand()
    {
        return !commandQueue.IsEmpty;
    }

    public ICommand Select()
    {
        if (!HasCommand())
        {
            return null;
        }

        var commands = commandQueue.ToArray();
        var command = commands[currentIndex];
        currentIndex = (currentIndex + 1) % commands.Length;
        return command;
    }

    public void Add(ICommand cmd)
    {
        commandQueue.Enqueue(cmd);
    }
}



In [7]:
public class WorkerThread
{
    private IScheduler scheduler;
    private CancellationTokenSource cancellationTokenSource;
    private CancellationToken cancellationToken;
    private Thread thread;

    public WorkerThread(IScheduler scheduler)
    {
        this.scheduler = scheduler;
        this.cancellationTokenSource = new CancellationTokenSource();
        this.cancellationToken = cancellationTokenSource.Token;
        this.thread = new Thread(Run);
    }

    public void Start()
    {
        thread.Start();
    }

    public void Stop()
    {
        cancellationTokenSource.Cancel();
        thread.Join();
    }

    private void Run()
    {
        while (!cancellationToken.IsCancellationRequested)
        {
            if (scheduler.HasCommand())
            {
                var command = scheduler.Select();
                if (command != null)
                {
                    if (!command.Execute())
                    {
                        scheduler.Add(command);
                    }
                }
            }
            else
            {
                Thread.Sleep(1);
            }
        }
    }
}